In [14]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/amex-cleaned-datasets/offer_metadata71.parquet
/kaggle/input/amex-cleaned-datasets/train_data_capped_cleaned.parquet
/kaggle/input/amex-cleaned-datasets/add_trans.parquet
/kaggle/input/amex-cleaned-datasets/agg_trans_features.parquet
/kaggle/input/amex-cleaned-datasets/add_event_cleaned.parquet
/kaggle/input/amex-cleaned-datasets/offermetadata_features.parquet
/kaggle/input/amex-cleaned-datasets/cleaned_test_dataa.parquet
/kaggle/input/amex-cleaned-datasets/engineered_add_event_features.parquet


In [15]:
df_events=pd.read_parquet('/kaggle/input/amex-cleaned-datasets/add_event_cleaned.parquet')
df_events


,id2,id3,id6,id4,id7
0,2431360,618619,Tiles,2023-10-22 08:08:17.768,None
1,2431360,363153,Tiles,2023-10-22 08:08:18.921,None
2,2431360,97193,Tiles,2023-10-22 08:08:17.765,None
3,2431360,654444,Tiles,2023-10-22 08:08:17.737,None
4,2431360,32325,Tiles,2023-10-22 08:08:17.812,None
...,...,...,...,...,...
21457468,2878066,197418,Tiles,2023-10-31 08:52:18.410,None
21457469,2878066,355406,Tiles,2023-10-31 08:52:17.470,None
21457470,2878066,146933,Tiles,2023-10-31 08:58:25.663,None
21457471,2878066,721756,Tiles,2023-10-31 08:58:25.497,None


In [16]:
df_events = df_events.replace({None: np.nan})
df_events

,id2,id3,id6,id4,id7
0,2431360,618619,Tiles,2023-10-22 08:08:17.768,NaN
1,2431360,363153,Tiles,2023-10-22 08:08:18.921,NaN
2,2431360,97193,Tiles,2023-10-22 08:08:17.765,NaN
3,2431360,654444,Tiles,2023-10-22 08:08:17.737,NaN
4,2431360,32325,Tiles,2023-10-22 08:08:17.812,NaN
...,...,...,...,...,...
21457468,2878066,197418,Tiles,2023-10-31 08:52:18.410,NaN
21457469,2878066,355406,Tiles,2023-10-31 08:52:17.470,NaN
21457470,2878066,146933,Tiles,2023-10-31 08:58:25.663,NaN
21457471,2878066,721756,Tiles,2023-10-31 08:58:25.497,NaN


In [17]:
column_name = 'id7'
null_count = df_events[column_name].isnull().sum()

print(f"Null values in '{column_name}':", null_count)

Null values in 'id7': 21058383


In [20]:


# Ensure only relevant columns are used
required_cols = ['id2', 'id3', 'id6', 'id4', 'id7']
df_events = df_events[required_cols].copy()

# Convert timestamps
df_events['id4'] = pd.to_datetime(df_events['id4'])  # Impression Timestamp
df_events['id7'] = pd.to_datetime(df_events['id7'])  # Click Timestamp

# Binary click flag
df_events['is_clicked'] = df_events['id7'].notna().astype(int)

# One-hot encode Placements ID (id6)
placement_dummies = pd.get_dummies(df_events['id6'], prefix='placement')
df_events = pd.concat([df_events, placement_dummies], axis=1)

# Time-based features
df_events['hour'] = df_events['id4'].dt.hour
df_events['day_of_week'] = df_events['id4'].dt.dayofweek
df_events['is_weekend'] = df_events['day_of_week'].isin([5, 6]).astype(int)

# Click delay in seconds
df_events['click_delay_seconds'] = (df_events['id7'] - df_events['id4']).dt.total_seconds()
df_events['clicked_within_60s'] = (df_events['click_delay_seconds'] <= 60).fillna(False).astype(int)
df_events['clicked_within_600s'] = (df_events['click_delay_seconds'] <= 600).fillna(False).astype(int)

# Customer-level aggregates
customer_agg = df_events.groupby('id2').agg(
    impressions_customer=('id3', 'count'),
    clicks_customer=('is_clicked', 'sum'),
    unique_offers_seen=('id3', 'nunique'),
    unique_placements_seen=('id6', 'nunique'),
)
customer_agg['customer_ctr'] = customer_agg['clicks_customer'] / customer_agg['impressions_customer']

# Offer-level aggregates
offer_agg = df_events.groupby('id3').agg(
    impressions_offer=('id2', 'count'),
    clicks_offer=('is_clicked', 'sum'),
)
offer_agg['offer_ctr'] = offer_agg['clicks_offer'] / offer_agg['impressions_offer']

# Placement-level aggregates
placement_agg = df_events.groupby('id6').agg(
    placement_impressions=('id2', 'count'),
    placement_clicks=('is_clicked', 'sum'),
)
placement_agg['placement_ctr'] = placement_agg['placement_clicks'] / placement_agg['placement_impressions']

# Merge all aggregates
df_events_features = df_events.merge(customer_agg, on='id2', how='left')
df_events_features = df_events_features.merge(offer_agg, on='id3', how='left')
df_events_features = df_events_features.merge(placement_agg, on='id6', how='left')

# Save
df_events_features.to_parquet('add_events_features.parquet', index=False)
